In [7]:
import pandas as pd 
import numpy as np
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score
import os 

In [8]:
# Groq API and Models 
Groq_Token = "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"  # Do not share this key with anyone

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}


1. Demonstrating zero shot learning and few shot learning.

Zero Shot Learning 

In [61]:
#zero shot learning 
categories = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
feature_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}


# Features
features_dataset = pd.read_csv("HAR/UCI HAR Dataset/features.txt", sep=" ", header=None)
Feature_names = features_dataset[1].tolist()
print(Feature_names[:15])

# Sample IMU data (replace these with actual values)
sample_data = pd.read_csv("HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep=r'\s+')
sample_labels = pd.read_csv("HAR/UCI HAR Dataset/test/y_test.txt",header=None,sep=r'\s+')
# Define the range of data to analyze
start_index = 0  
end_index = 5
#Taking part of sample data 

Acc_Data = sample_data.iloc[start_index:end_index,:15]
subset_data = Acc_Data.values.tolist()

True_labels = sample_labels.iloc[start_index:end_index].values.flatten().tolist()
mapped_true_labels = [list(feature_mapping.keys())[list(feature_mapping.values()).index(label)] for label in True_labels]


# print(subset_data)
# print(True_labels)
# print(len(subset_data))

# Initialize an empty list to hold the results
activity_labels = []

for data in subset_data:
    # System Prompt
    query = f"""
    * You are a human activity analysis model.
    * You are given data :{data} which have features : {Feature_names[:15]}
    * Given that the person's activity must be one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING,
    Provide the activity labels ONLY in a space-separated format as the output. Do not provide any other text.
    """
    
    # Invoke the LLM
    model_name = "llama3-70b"  # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    # Collect the results
    print(data)
    activity_labels.append(answer.content.strip())

# Print the results
print(activity_labels)
print(mapped_true_labels)

print(accuracy_score(activity_labels,mapped_true_labels))


['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z']
[0.25717778, -0.02328523, -0.014653762, -0.938404, -0.92009078, -0.66768331, -0.95250112, -0.92524867, -0.67430222, -0.89408755, -0.55457721, -0.46622295, 0.71720847, 0.6355024, 0.78949666]
[0.28602671, -0.013163359, -0.11908252, -0.97541469, -0.9674579, -0.94495817, -0.9867988, -0.96840133, -0.9458234, -0.89408755, -0.55457721, -0.80601325, 0.76803131, 0.68369799, 0.79670578]
[0.27548482, -0.02605042, -0.11815167, -0.99381904, -0.96992551, -0.96274798, -0.99440345, -0.97073498, -0.96348267, -0.93926027, -0.56851222, -0.79911648, 0.84830517, 0.66786423, 0.82244227]
[0.27029822, -0.032613869, -0.11752018, -0.99474279, -0.97326761, -0.96709068, -0.99527433, -0.97447101, -0.96889736, -0.938

Few Shot Learning

In [96]:
#zero shot learning 
categories = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
feature_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}


# Features
features_dataset = pd.read_csv("HAR/UCI HAR Dataset/features.txt", sep=" ", header=None)
Feature_names = features_dataset[1].tolist()
print(Feature_names[:15])

# Sample IMU data (replace these with actual values)
sample_data = pd.read_csv("HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep=r'\s+')
sample_labels = pd.read_csv("HAR/UCI HAR Dataset/test/y_test.txt",header=None,sep=r'\s+')
# Define the range of data to analyze
start_index = 0  
end_index = 30
#Taking part of sample data 

Acc_Data = sample_data.iloc[start_index:end_index,:15]
subset_data = Acc_Data.values.tolist()

True_labels = sample_labels.iloc[start_index:end_index].values.flatten().tolist()
mapped_true_labels = [list(feature_mapping.keys())[list(feature_mapping.values()).index(label)] for label in True_labels]


# print(subset_data)
# print(True_labels)
# print(len(subset_data))

# Storing and formatting Examples
Example_Data = pd.read_csv("HAR/UCI HAR Dataset/train/X_train.txt", header = None,sep= r'\s+')
Example_Data_labels = pd.read_csv("HAR/UCI HAR Dataset/train/y_train.txt", header = None,sep= r'\s+')
formatted_examples = []
for n in range(50):  
    
    example_acc_data = Example_Data.iloc[n].tolist()
    label = categories[Example_Data_labels.iloc[n, 0] - 1]
    formatted_example = f"Example {n+1}:\n" \
                        f"Acceleration Data: {example_acc_data[:15]}\n" \
                        f"Label: {label}\n"
    formatted_examples.append(formatted_example)

examples_string = "\n".join(formatted_examples)

print(examples_string)

# Initialize an empty list to hold the results
activity_labels = []

for data in subset_data:
    # System Prompt
    query = f"""
    * You are a human activity analysis model.
    * Examples: {examples_string}
    * You are given data :{data} which have features : {Feature_names[:15]}
    * Given that the person's activity must be one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING,
    Provide the activity labels ONLY in a space-separated format as the output. Do not provide any other text.
    """
    
    # Invoke the LLM
    model_name = "llama3-70b"  # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    # Collect the results
    print(data)
    activity_labels.append(answer.content.strip())

# Print the results
print(activity_labels)
print(mapped_true_labels)

print(accuracy_score(activity_labels,mapped_true_labels))


['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z']
Example :
Acceleration Data: [0.28858451, -0.020294171, -0.13290514, -0.9952786, -0.98311061, -0.91352645, -0.99511208, -0.98318457, -0.92352702, -0.93472378, -0.56737807, -0.74441253, 0.85294738, 0.68584458, 0.81426278]
Label: STANDING

Example :
Acceleration Data: [0.27841883, -0.016410568, -0.12352019, -0.99824528, -0.97530022, -0.96032199, -0.99880719, -0.97491437, -0.95768622, -0.94306751, -0.55785126, -0.81840869, 0.84930787, 0.68584458, 0.82263681]
Label: STANDING

Example :
Acceleration Data: [0.27965306, -0.019467156, -0.11346169, -0.99537956, -0.96718701, -0.97894396, -0.99651994, -0.96366837, -0.97746859, -0.93869155, -0.55785126, -0.81840869, 0.84360895, 0.68240094, 0.8393441

Q1. Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data. Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why? 

Few Shot learning is more better then Zero shot as in zero shot learning it predicts the activty based on guess whereas in few_shot_learning as examples are provided it compares the activity data and gives predictions

Q2. Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why?

Decision trees is better then Few_shot Learning as accuray_score of decision tree is 0.74 and as compared to Few shot learning which has accrucy_score of 0.625 . Also the Accuracy of Few shot learning is not the same stable .while performing the test on few shot we got accarcy as 0.4 ,0.6 ,0.8 ,0.625 which shows that the method is not better then Decision trees.

Q3. What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data?

Zero shot learning may have bias towards seen data and it is thus not able to classify for other activity and for Few-shot learning if it is given data for only 1 or 2 activity it may overfit .In general Zero shot makes guesses making it poor model. Both of them are api calls which have limitations thus we are unable to provide large data to them making.

Q4.What does the model classify when given input from an entirely new activity that it hasn't seen before? 

For this we are removing Standing data from the dataset and testing for 5 sample data. for Zero shot Learning it will just make Guess since it does not have examples.Thus predciton on new data is same for zero shot learning.

now for Few shot learning it gives accruacy zero also it is not able to predict STANDING nut it was able to classify Laying similar to STANDING form the given ouput 

In [102]:
#zero shot learning 
categories = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
feature_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}


# Features
features_dataset = pd.read_csv("HAR/UCI HAR Dataset/features.txt", sep=" ", header=None)
Feature_names = features_dataset[1].tolist()
print(Feature_names[:15])

# Sample IMU data (replace these with actual values)
sample_data = pd.read_csv("HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep=r'\s+')
sample_labels = pd.read_csv("HAR/UCI HAR Dataset/test/y_test.txt",header=None,sep=r'\s+')
# Define the range of data to analyze
start_index = 0  
end_index = 8
#Taking part of sample data 

Acc_Data = sample_data.iloc[start_index:end_index,:15]
subset_data = Acc_Data.values.tolist()

True_labels = sample_labels.iloc[start_index:end_index].values.flatten().tolist()
mapped_true_labels = [list(feature_mapping.keys())[list(feature_mapping.values()).index(label)] for label in True_labels]


# print(subset_data)
# print(True_labels)
# print(len(subset_data))

# Storing and formatting Examples
Example_Data = pd.read_csv("HAR/UCI HAR Dataset/train/X_train.txt", header = None,sep= r'\s+')
Example_Data_labels = pd.read_csv("HAR/UCI HAR Dataset/train/y_train.txt", header = None,sep= r'\s+')

# For each label we are trying to add examples and not adding examples for Label : STANDING 
formatted_examples = []
label_count = {label: 0 for label in categories}  

for n in range(500):  
    label_index = Example_Data_labels.iloc[n, 0] - 1
    label = categories[label_index]
    
    if label_count[label] < 10 and label != "STANDING":  
        example_acc_data = Example_Data.iloc[n].tolist()
        formatted_example = f"Example :\n" \
                            f"Acceleration Data: {example_acc_data[:15]}\n" \
                            f"Label: {label}\n"
        formatted_examples.append(formatted_example)
        label_count[label] += 1  
        
    if all(count == 2 for count in label_count.values()): 
        break

examples_string = "\n".join(formatted_examples)

print(examples_string)


# Initialize an empty list to hold the results
activity_labels = []

for data in subset_data:
    # System Prompt
    query = f"""
    * You are a human activity analysis model.
    * Examples: {examples_string}
    * You are given data :{data} which have features : {Feature_names[:15]}
    * Given that the person's activity must be one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING,
    Provide the activity labels ONLY in a space-separated format as the output. Do not provide any other text.
    """
    
    # Invoke the LLM
    model_name = "llama3-70b"  # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    # Collect the results
    print(data)
    activity_labels.append(answer.content.strip())

# Print the results
print(activity_labels)
print(mapped_true_labels)

print(accuracy_score(activity_labels,mapped_true_labels))





['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z']
Example :
Acceleration Data: [0.14450396, 0.18926326, 0.062769317, -0.90429967, -0.18193654, -0.44315051, -0.90110019, -0.11081268, -0.40059935, -0.93189578, 0.042098681, -0.3365258, 0.71647623, 0.65581306, 0.80898559]
Label: SITTING

Example :
Acceleration Data: [0.28725164, -0.037455064, -0.14597431, -0.98291504, -0.89160489, -0.94143811, -0.98441781, -0.89137296, -0.93336072, -0.93189578, -0.49953681, -0.82469972, 0.82636555, 0.65581306, 0.80898559]
Label: SITTING

Example :
Acceleration Data: [0.2799976, -0.019484036, -0.10572355, -0.99281839, -0.94035041, -0.98149266, -0.99309238, -0.93692706, -0.9806691, -0.935389, -0.56525476, -0.82253173, 0.84772493, 0.66274101, 0.83760928]
Label

5. Few_Shot leanring with random data

In [ ]:
#testing Zero shot learning model on Random Data 
#zero shot learning 
categories = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
feature_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}


# Features
features_dataset = pd.read_csv("HAR/UCI HAR Dataset/features.txt", sep=" ", header=None)
Feature_names = features_dataset[1].tolist()
print(Feature_names[:15])

# Sample IMU data (replace these with actual values)
sample_data = pd.read_csv("HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep=r'\s+')
sample_labels = pd.read_csv("HAR/UCI HAR Dataset/test/y_test.txt",header=None,sep=r'\s+')
# # Define the range of data to analyze
# start_index = 0  
# end_index = 5

#Making Random Sample data for testing 

for i in range(5):
    n = np.rand.Random(1000)
    Acc_Data = sample_data.iloc[start_index:end_index,:15]
    subset_data = Acc_Data.values.tolist()
    



True_labels = sample_labels.iloc[start_index:end_index].values.flatten().tolist()
mapped_true_labels = [list(feature_mapping.keys())[list(feature_mapping.values()).index(label)] for label in True_labels]


# print(subset_data)
# print(True_labels)
# print(len(subset_data))

# Initialize an empty list to hold the results
activity_labels = []

for data in subset_data:
    # System Prompt
    query = f"""
    * You are a human activity analysis model.
    * You are given data :{data} which have features : {Feature_names[:15]}
    * Given that the person's activity must be one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING,
    Provide the activity labels ONLY in a space-separated format as the output. Do not provide any other text.
    """
    
    # Invoke the LLM
    model_name = "llama3-70b"  # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    # Collect the results
    print(data)
    activity_labels.append(answer.content.strip())

# Print the results
print(activity_labels)
print(mapped_true_labels)

print(accuracy_score(activity_labels,mapped_true_labels))


////////////////////////////////////

In [91]:
#zero shot learning 
categories = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]
feature_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}


# Features
features_dataset = pd.read_csv("HAR/UCI HAR Dataset/features.txt", sep=" ", header=None)
Feature_names = features_dataset[1].tolist()
print(Feature_names[:15])

# Sample IMU data (replace these with actual values)
sample_data = pd.read_csv("HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep=r'\s+')
sample_labels = pd.read_csv("HAR/UCI HAR Dataset/test/y_test.txt",header=None,sep=r'\s+')
# Define the range of data to analyze
start_index = 0  
end_index = 8
#Taking part of sample data 

Acc_Data = sample_data.iloc[start_index:end_index,:15]
subset_data = Acc_Data.values.tolist()

True_labels = sample_labels.iloc[start_index:end_index].values.flatten().tolist()
mapped_true_labels = [list(feature_mapping.keys())[list(feature_mapping.values()).index(label)] for label in True_labels]


# print(subset_data)
# print(True_labels)
# print(len(subset_data))

# Storing and formatting Examples
Example_Data = pd.read_csv("HAR/UCI HAR Dataset/train/X_train.txt", header = None,sep= r'\s+')
Example_Data_labels = pd.read_csv("HAR/UCI HAR Dataset/train/y_train.txt", header = None,sep= r'\s+')

# For each label we are trying to add 2 examples 
formatted_examples = []
label_count = {label: 0 for label in categories}  

for n in range(500):  
    label_index = Example_Data_labels.iloc[n, 0] - 1
    label = categories[label_index]
    
    if label_count[label] < 10:  
        example_acc_data = Example_Data.iloc[n].tolist()
        formatted_example = f"Example :\n" \
                            f"Acceleration Data: {example_acc_data[:15]}\n" \
                            f"Label: {label}\n"
        formatted_examples.append(formatted_example)
        label_count[label] += 1  
        
    if all(count == 2 for count in label_count.values()): 
        break

examples_string = "\n".join(formatted_examples)

print(examples_string)


# Initialize an empty list to hold the results
activity_labels = []

for data in subset_data:
    # System Prompt
    query = f"""
    * You are a human activity analysis model.
    * Examples: {examples_string}
    * You are given data :{data} which have features : {Feature_names[:15]}
    * Given that the person's activity must be one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING,
    Provide the activity labels ONLY in a space-separated format as the output. Do not provide any other text.
    """
    
    # Invoke the LLM
    model_name = "llama3-70b"  # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    # Collect the results
    print(data)
    activity_labels.append(answer.content.strip())

# Print the results
print(activity_labels)
print(mapped_true_labels)

print(accuracy_score(activity_labels,mapped_true_labels))


['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z']
Example :
Acceleration Data: [0.28858451, -0.020294171, -0.13290514, -0.9952786, -0.98311061, -0.91352645, -0.99511208, -0.98318457, -0.92352702, -0.93472378, -0.56737807, -0.74441253, 0.85294738, 0.68584458, 0.81426278]
Label: STANDING

Example :
Acceleration Data: [0.27841883, -0.016410568, -0.12352019, -0.99824528, -0.97530022, -0.96032199, -0.99880719, -0.97491437, -0.95768622, -0.94306751, -0.55785126, -0.81840869, 0.84930787, 0.68584458, 0.82263681]
Label: STANDING

Example :
Acceleration Data: [0.27965306, -0.019467156, -0.11346169, -0.99537956, -0.96718701, -0.97894396, -0.99651994, -0.96366837, -0.97746859, -0.93869155, -0.55785126, -0.81840869, 0.84360895, 0.68240094, 0.8393441